# Scraping Hotel Ratings on Booking # 

In this homework we will practice web scraping on the following [site](https://www.booking.com/searchresults.html?label=gen173nr-1DCAEoggJCAlhYSDNYBHIFdXNfbWGIAQGYATG4AQfIAQ3YAQPoAQH4AQKSAgF5qAID&sid=991709bb93b40898e2ebcc9f2e20ade5&class_interval=1&dest_id=20088325&dest_type=city&from_sf=1&group_adults=2&group_children=0&label_click=undef&nflt=di%3D929%3B&no_rooms=1&pop_filter_id=di-929&pop_filter_pos=0&pop_filter_rank=2&raw_dest_type=city&room1=A%2CA&sb_price_type=total&search_selected=1&src=index&src_elem=sb&ss=New%20York%2C%20New%20York%20State%2C%20USA&ss_raw=new%20york&ssb=empty). Let's get some basic information for **each hotel in Manhattan (follow the above link) **.
On each hotel page, scrape the following information: 
1. Hotel Name
2. Class of Rating (Wonderful/Excellent/Very Good/Good)
3. Rating Score

** Save the data in "hotel_information.csv" in the following format: hotel_name, class_rating, rating **

**(4 pts)**

Now let's scrape some reviews. You can see all review information by going to the hotel page and accessing on “see all reviews” and then view the page sourceFor each review of each each hotel in Manhattan you are to scrape the following attributes: 
1. Hotel name
2. Reviewer name
3. Number of helpful votes (if available, otherwise None)
4. Date
5. Review Rating
6. Negative Review (if available, otherwise None)
7. Positive Review (if available, otherwise None)

Note that you will also need the hotel's name!! Also, some reviews may not have all attributes so then just add None in the corresponding column. 

** Save the data in "review_ratings.csv" in the following format: hotel_name, reviewer_name, num_help_votes, date, review_rating, neg_review, pos_review **

**(4 pts)**

In [1]:
import requests

url = "https://www.booking.com/searchresults.html?label=gen173nr-1DCAEoggJCAlhYSDNYBHIFdXNfbWGIAQGYATG4AQfIAQ3YAQPoAQH4AQKSAgF5qAID;sid=79b79e55b2cbdbbe0fcfdb6fc3c1402f;class_interval=1&dest_id=20088325&dest_type=city&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&nflt=di%3D929%3B&no_rooms=1&offset=0&postcard=0&raw_dest_type=city&room1=A%2CA&sb_price_type=total&search_selected=1&src=index&src_elem=sb&ss=New%20York%2C%20New%20York%20State%2C%20USA&ss_all=0&ss_raw=new%20york&ssb=empty&sshis=0&"
#url = "https://www.booking.com/searchresults.zh-cn.html?label=gen173nr-1DCAEoggJCAlhYSDNYBHIFdXNfbWGIAQGYATG4AQfIAQ3YAQPoAQH4AQKSAgF5qAID;sid=79b79e55b2cbdbbe0fcfdb6fc3c1402f;class_interval=1&dest_id=20088325&dest_type=city&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&nflt=di%3D929%3B&no_rooms=1&offset=0&postcard=0&raw_dest_type=city&room1=A%2CA&sb_price_type=total&search_selected=1&src=index&src_elem=sb&ss=New%20York%2C%20New%20York%20State%2C%20USA&ss_all=0&ss_raw=new%20york&ssb=empty&sshis=0&"

In [2]:
response = requests.get(url)
html = response.text.encode('utf-8')

In [3]:
from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import argparse
import codecs
import json
import re
import csv
from pandas.core.frame import DataFrame

soup = BeautifulSoup(html, "html5lib")

In [4]:
def scrape_page(html,objs):
    soup = BeautifulSoup(html, "html5lib")
    AllDiv = soup.findAll("div",{'class' :'sr_item_default'})
    for span in AllDiv:
        obj = []
        try:
            name = span.find('span',{'class':"sr-hotel__name"}).text.strip('\n')
        except Exception as e:
            name = "None"
            rate = "None"
            _class = "None"
        try:
            rate = span.find('span',{'class':"review-score-badge"}).text.strip('\n')
        except Exception as e:
            rate = "None"
        try:
            _class = span.find('span',{'class':"review-score-widget__text"}).text.strip('\n')
        except Exception as e:
            _class = "None"
        try:
            #href = span.find('a',{'class':"hotel_name_link url"})['href'].split("#")[0].strip('\n')
            href = span.find('a',{'class':"sr_item_photo_link"})['href']
        except Exception as e:
            href = "None"
            
        obj = [name,_class,rate]
        objs.append(obj)
        hrefs.append(href)
        #print(obj)
        
        
    
    try:
        _next = soup.find("a",{'class':'paging-next'})
        #print(_next.text)
        if _next.text == "Next page" :
            url = _next['href']
            response = requests.get(url)
            html = response.text.encode('utf-8')
            return scrape_page(html,objs)
        else:
            return objs
    except Exception as e:
        return objs

In [5]:
tmp = []
hrefs = []
objs = scrape_page(html,tmp)

df = DataFrame(objs,columns = ["hotel_name","Class of Rating", "Rating"])

df

,hotel_name,Class of Rating,Rating
0,Tribeca Lofts,Good,7.9
1,LUMA Hotel - Times Square,Awesome,9.2
2,Amazing Apartment- Walking Distance to Times SQ,Very Good,8.3
3,Fairfield Inn & Suites by Marriott New York Do...,Excellent,8.9
4,Splendid Apartment by Times SQ,Very Good,8.2
5,Radio City Apartments,Very Good,8.5
6,La Quinta Inn & Suites Manhattan,Very Good,8.0
7,Hotel 50 Bowery,Awesome,9.2
8,Archer Hotel New York,Awesome,9.0
9,Freehand New York,Awesome,9.1


In [6]:
print(hrefs)

['/hotel/us/tribeca-lofts.html', '/hotel/us/hotel-lower-east-side-new-york.html', '/hotel/us/ultra-luxurious-designer-large-appartment-top-floor-fitness-doorman-terrace-j.html', '/hotel/us/radio-city-apartments.html', '/hotel/us/50-bowery.html', '/hotel/us/freehand-new-york.html', '/hotel/us/archer-new-york.html', '/hotel/us/aka-times-square.html', '/hotel/us/times-square-lux-condo.html', '/hotel/us/gramercy-townhouse.html', '/hotel/us/comfort-inn-midtown.html', '/hotel/us/lombardy-manhattan.html', '/hotel/us/chambers.html', '/hotel/us/korman-kommunities-central-park.html', '/hotel/us/two-bedroom-apartment-upper-west-side.html', '/hotel/us/lombardy-manhattan.html', '/hotel/us/nolitan.html', '/hotel/us/korman-kommunities-central-park.html', '/hotel/us/two-bedroom-apartment-upper-west-side.html', '/hotel/us/beautiful-2-bdr-roof-1-block-from-central-park.html', '/hotel/us/crowne-plaza-times-square.html', '/hotel/us/the-allen.html', '/hotel/us/trump-soho.html', '/hotel/us/midtown-east-spac

In [11]:
df.to_csv('C:\software\cs 506\cs506-spring2018-hw3-jweixuan\hotel_information.csv', encoding = 'utf-8')

In [6]:
def next_review_page(html,name,each_comment):
    response_review = requests.get(html)
    html_review = response_review.text.encode('utf-8')
    soup_review = BeautifulSoup(html_review, "html5lib")
    
    li = soup_review.findAll('li',{'class':'review_item clearfix '})
    for i in li:
        reviewer = i.find('span',{'itemprop':'name'}).text.strip('\n')
        date = i.find('p',{'class':'review_item_date'}).text.strip('\n')
        score = i.find('span',{'class':'review-score-badge'}).text.strip('\n')
        
        try:
            reviews_pos = i.find('p',{'class':'review_pos'})
            Positive_Review = reviews_pos.find('span',{'itemprop':'reviewBody'}).text.strip('\n')
        except Exception as e:
            Positive_Review = "None"
        try:
            reviews_neg = i.find('p',{'class':'review_neg'})
            Negative_Review = reviews_neg.find('span',{'itemprop':'reviewBody'}).text.strip('\n')
        except Exception as e:
            Negative_Review = "None"
            
        #print([name,reviewer,date,score,Positive_Review,Negative_Review])
        each_comment.append([name,reviewer,date,score,Positive_Review,Negative_Review])
        
    try:
        next_href = soup_review.find('p',{'class':'review_next_page'}).find('a')['href']
        next_href = "https://www.booking.com" + next_href
        print(next_href)
        #response = requests.get(url)
        #html = response.text.encode('utf-8')
        return next_review_page(next_href,name,each_comment)
    except Exception as e:
        return  each_comment    #modified to list

In [7]:
def see_all_reviews(html,each_comment):
    soup = BeautifulSoup(html, "html5lib")
    name = soup.find('h2',{'class':'hp__hotel-name'}).text.strip('\n')
    try:
        review_href = "https://www.booking.com" + soup.find('a',{'class':'show_all_reviews_btn'})['href']
        print("review:  ",review_href)
    except Exception as e:
        each_comment.append([name,'None','None','None','None','None'])
        return each_comment   #queshao tiaojian   !!!!!!!!!!!
    next_review_page(review_href,name,each_comment)
    return each_comment


In [8]:
#df2 = DataFrame(columns = ["Hotel_name",'Date','Score','Positive_Review','Negative_Review'])
k = 2
each_comment = []
for i in hrefs:
    #each_comment = []
    url = "https://www.booking.com" + i
    print("hrefs:  ",url)
    response = requests.get(url)
    html = response.text.encode('utf-8')
    each_comment = see_all_reviews(html,each_comment)
    #df2.loc[df.shape[0]+1] = {"Hotel_name":each_comment[0],'Date':each_comment[1],'Score':each_comment[2],'Positive_Review':each_comment[3],'Negative_Review':each_comment[4]}
    #df2 = DataFrame(each_comment,columns = ["Hotel_name",'Date','Score','Positive_Review','Negative_Review'])
#     if k == 0:
#         break
#     else:
#         k -= 1

hrefs:   https://www.booking.com/hotel/us/tribeca-lofts.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/tribeca-lofts.html
hrefs:   https://www.booking.com/hotel/us/luma-times-square.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/luma-times-square.html
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=2&
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=3&
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=4&
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=5&
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=6&
https://www.booking.com/reviews/us/hotel/luma-times-square.html?page=7&
hrefs:   https://www.booking.com/hotel/us/splendid-apartments-times-square-manhattan.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/splendid-apartments-times-square-manhattan.html
https://www.booking.com/reviews/us/ho

review:   https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=2&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=3&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=4&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=5&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=6&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=7&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=8&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=9&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=10&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=11&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-plaza.html?page=12&
https://www.booking.com/reviews/us/hotel/new-yorker-ramada-

https://www.booking.com/reviews/us/hotel/hudson.html?page=55&
https://www.booking.com/reviews/us/hotel/hudson.html?page=56&
https://www.booking.com/reviews/us/hotel/hudson.html?page=57&
https://www.booking.com/reviews/us/hotel/hudson.html?page=58&
hrefs:   https://www.booking.com/hotel/us/heart-of-midtown.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com#blockdisplay4
hrefs:   https://www.booking.com/hotel/us/two-bedroom-apartment-upper-west-side.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/korman-kommunities-central-park.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/korman-kommunities-central-park.html
https://www.booking.com/reviews/us/hotel/korman-kommunities-central-park.html?page=2&
https://www.booking.com/reviews/us/hotel/korman-kommunities-central-park.html?page=3&
https://www.booking.com/reviews/us/hotel/korman-kommunities-central-park.html?page=4&
https://www.booking.com/reviews/us/hotel/korman-kommunitie

hrefs:   https://www.booking.com/hotel/us/the-william-extended-stay.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/the-william-extended-stay.html
https://www.booking.com/reviews/us/hotel/the-william-extended-stay.html?page=2&
https://www.booking.com/reviews/us/hotel/the-william-extended-stay.html?page=3&
https://www.booking.com/reviews/us/hotel/the-william-extended-stay.html?page=4&
https://www.booking.com/reviews/us/hotel/the-william-extended-stay.html?page=5&
hrefs:   https://www.booking.com/hotel/us/the-new-york-at-times-square.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/the-new-york-at-times-square.html
https://www.booking.com/reviews/us/hotel/the-new-york-at-times-square.html?page=2&
https://www.booking.com/reviews/us/hotel/the-new-york-at-times-square.html?page=3&
https://www.booking.com/reviews/us/hotel/the-new-york-at-times-square.html?page=4&
https://www.booking.com/reviews/us/hotel/the-new-york-at-times-squa

https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=9&
https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=10&
https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=11&
https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=12&
https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=13&
https://www.booking.com/reviews/us/hotel/newyork-510-west-42.html?page=14&
hrefs:   https://www.booking.com/hotel/us/john-street-apartment.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/sanctuary-new-york.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/sanctuary-new-york.html
https://www.booking.com/reviews/us/hotel/sanctuary-new-york.html?page=2&
https://www.booking.com/reviews/us/hotel/sanctuary-new-york.html?page=3&
https://www.booking.com/reviews/us/hotel/sanctuary-new-york.html?page=4&
https://www.booking.com/reviews/us/hotel/sanctuary-new-york.html?page=5&
h

...

hrefs:   https://www.booking.com/hotel/us/the-night.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/the-night.html
https://www.booking.com/reviews/us/hotel/the-night.html?page=2&
https://www.booking.com/reviews/us/hotel/the-night.html?page=3&
https://www.booking.com/reviews/us/hotel/the-night.html?page=4&
https://www.booking.com/reviews/us/hotel/the-night.html?page=5&
https://www.booking.com/reviews/us/hotel/the-night.html?page=6&
https://www.booking.com/reviews/us/hotel/the-night.html?page=7&
https://www.booking.com/reviews/us/hotel/the-night.html?page=8&
hrefs:   https://www.booking.com/hotel/us/west-46th-street-apt-4c-4rw.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/mont-morris-house.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/mont-morris-house.html
https://www.booking.com/reviews/us/hotel/mont-morris-house.html?page=2&
hrefs:   https://www.booking.com/hotel/us/permago-wall-street.html?from

hrefs:   https://www.booking.com/hotel/us/heart-of-lower-manhattan.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/beautiful-midtown-studio.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/oakwood-6th-avenue.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/colonial-house-inn.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/colonial-house-inn.html
https://www.booking.com/reviews/us/hotel/colonial-house-inn.html?page=2&
https://www.booking.com/reviews/us/hotel/colonial-house-inn.html?page=3&
https://www.booking.com/reviews/us/hotel/colonial-house-inn.html?page=4&
hrefs:   https://www.booking.com/hotel/us/two-bedroom-apartment-union-square.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/two-bedroom-apartment-union-square.html
hrefs:   https://www.booking.com/hotel/us/w-new-york-union-square.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/

hrefs:   https://www.booking.com/hotel/us/holiday-inn-express-manhattan-midtown-west.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/holiday-inn-express-manhattan-midtown-west.html
https://www.booking.com/reviews/us/hotel/holiday-inn-express-manhattan-midtown-west.html?page=2&
hrefs:   https://www.booking.com/hotel/us/wyndam-garden-chinatown.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=2&
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=3&
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=4&
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=5&
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=6&
https://www.booking.com/reviews/us/hotel/wyndam-garden-chinatown.html?page=7&
hrefs:   https://www.booking.com/hotel/us/taste

review:   https://www.booking.com/reviews/us/hotel/redford-hotel-new-york.html
https://www.booking.com/reviews/us/hotel/redford-hotel-new-york.html?page=2&
hrefs:   https://www.booking.com/hotel/us/midtown-best-location-28th-street.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com#blockdisplay4
hrefs:   https://www.booking.com/hotel/us/nyc-aparment-for-rent.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/studio-times-square-new-york.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com#blockdisplay4
hrefs:   https://www.booking.com/hotel/us/midtown-2br-by-central-park.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/midtown-2br-by-central-park.html
hrefs:   https://www.booking.com/hotel/us/luxurious-three-bedroom-apartment.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com#blockdisplay4
hrefs:   https://www.booking.com/hotel/us/superior-midtown-east-apartments.html?from_hc_img=1#hotelTmpl
review:   

https://www.booking.com/reviews/us/hotel/city-rooms-nyc.html?page=4&
hrefs:   https://www.booking.com/hotel/us/cozy-studio-esb.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/cozy-studio-esb.html
hrefs:   https://www.booking.com/hotel/us/eclectic-2bedroom-art-studio.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/28th-street-amp-park-avenue-apartments.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/hamilton-heights-beauty.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/hamilton-heights-beauty.html
hrefs:   https://www.booking.com/hotel/us/apartment-new-york-new-york.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/three-bedroom-apartment-with-city-view-lincoln-center-new-york.html?from_hc_img=1#hotelTmpl
hrefs:   https://www.booking.com/hotel/us/the-grand-apartment.html?from_hc_img=1#hotelTmpl
review:   https://www.booking.com/reviews/us/hotel/the-grand-ap

In [10]:
df2 = DataFrame(each_comment,columns = ["Hotel_name",'Reviewer_name','Date','Review_Rating','Positive_Review','Negative_Review'])
df2

,Hotel_name,Reviewer_name,Date,Review_Rating,Positive_Review,Negative_Review
0,Tribeca Lofts,Ryo,"Reviewed: March 29, 2018",7.9,Nice and large flat conveniently located withi...,"As of March 27th, 2018, the building was under..."
1,Tribeca Lofts,Matthew,"Reviewed: March 21, 2018",10,What a great experience from the check in till...,None
2,Tribeca Lofts,Laurent,"Reviewed: March 8, 2018",8.3,"Excellent location, large, well equiped, and n...","We booked for 6, but we had bed sheets for 4, ..."
3,Tribeca Lofts,Nicolas,"Reviewed: February 22, 2018",10,"Amazing place! Angie did our check in, she was...",None
4,Tribeca Lofts,Nat,"Reviewed: February 22, 2018",10,Best experience I have ever had with a service...,None
5,Tribeca Lofts,Pierre,"Reviewed: December 29, 2017",3.3,None,"We made a request for 6 guests, 2 adults 3 tee..."
6,Tribeca Lofts,Todd,"Reviewed: March 29, 2018",10,feeling of openness/cozy,the couch needs upgrade
7,LUMA Hotel - Times Square,Ash,"Reviewed: March 29, 2018",9.6,Excellent!,I needed a late check out and was only offered...
8,LUMA Hotel - Times Square,Karen,"Reviewed: March 28, 2018",10,This property is fantastic! Amazing location! ...,Absolutely nothing 😊
9,LUMA Hotel - Times Square,Catherine,"Reviewed: March 24, 2018",10,King studio was modern and spacious. It had ev...,None


In [13]:
df2.to_csv('C:\\software\\cs 506\\cs506-spring2018-hw3-jweixuan\\review_rating.csv', encoding = 'utf-8')